In [4]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

### <font color='black'> Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!</font>

In [53]:
with engine.connect() as con:
    rs = con.execute("SELECT f.name AS facilityname, \
                     SUM(CASE WHEN b.memid = 0 THEN b.slots*f.guestcost \
                        ELSE b.slots*f.membercost END) AS revenue \
                     FROM Facilities AS f \
                     INNER JOIN Bookings AS b \
                     ON b.facid = f.facid \
                     GROUP BY facilityname \
                     HAVING revenue < 1000 \
                     ORDER BY revenue DESC;")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
print(df)

    facilityname  revenue
0     Pool Table      270
1  Snooker Table      240
2   Table Tennis      180


### <font color='black'> Produce a report of members and who recommended them in alphabetic surname, firstname order</font>

In [64]:
with engine.connect() as con:
    rs = con.execute("SELECT m.surname || ', ' || m.firstname AS member, \
                     r.surname || ', ' || r.firstname AS recommendedby \
                     FROM Members AS m \
                     LEFT JOIN Members AS r \
                     ON m.recommendedby = r.memid \
                     ORDER BY member;")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
print(df)

                      member       recommendedby
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8             Farrell, David                None
9            Farrell, Jemima                None
10              GUEST, GUEST                None
11          Genting, Matthew     Butters, Gerald
12                Hunt, John  Purview, Millicent
13              Jones, David    Joplette, Janice
14            Jones, Douglas        Jones, David
15          Joplette, Janice       Smith, Darren
16           Mackenzie, Anna       Smith, Darren
17             Owen, Charles       Smith, Darren
18             Pinker, David     Farrell, Jemima
19        Purview, M

### <font color='black'> Find the facilities with their usage by member, but not guests</font>

In [76]:
with engine.connect() as con:
    rs = con.execute("SELECT m.firstname || ' ' || m.surname AS membername, \
                      f.name AS facilityname, \
                      COUNT(b.bookid) AS usage \
                      FROM Facilities AS f \
                      INNER JOIN Bookings AS b \
                      ON f.facid = b.facid \
                      INNER JOIN Members AS m \
                      ON b.memid = m.memid \
                      WHERE b.memid != 0 \
                      GROUP BY membername, facilityname \
                      ORDER BY usage DESC")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
print(df)

            membername     facilityname  usage
0           Tim Rownam       Pool Table    241
1         Darren Smith  Badminton Court    132
2        Timothy Baker       Pool Table     85
3           Tim Rownam   Massage Room 1     80
4       Anna Mackenzie       Pool Table     70
..                 ...              ...    ...
197    Matthew Genting   Tennis Court 1      1
198  Millicent Purview    Snooker Table      1
199  Millicent Purview     Squash Court      1
200  Millicent Purview   Tennis Court 2      1
201    Ponder Stibbons   Tennis Court 1      1

[202 rows x 3 columns]


### <font color='black'> Find the facilities usage by month, but not guests</font>

In [77]:
with engine.connect() as con:
    rs = con.execute("SELECT m.firstname || ' ' || m.surname AS membername, \
                      f.name AS facilityname, \
                      strftime('%m',b.starttime) AS month, \
                      COUNT(b.bookid) AS usage \
                      FROM Facilities AS f \
                      INNER JOIN Bookings AS b \
                      ON f.facid = b.facid \
                      INNER JOIN Members AS m \
                      ON b.memid = m.memid \
                      WHERE b.memid != 0 \
                      GROUP BY month, facilityname \
                      ORDER BY month ASC, usage DESC")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
print(df)

         membername     facilityname month  usage
0      Darren Smith       Pool Table    07    103
1      Darren Smith   Massage Room 1    07     77
2      Darren Smith    Snooker Table    07     68
3       Tracy Smith   Tennis Court 1    07     65
4      Darren Smith  Badminton Court    07     51
5      Darren Smith     Table Tennis    07     48
6        Tim Rownam   Tennis Court 2    07     41
7      Darren Smith     Squash Court    07     23
8   Janice Joplette   Massage Room 2    07      4
9        Tim Rownam       Pool Table    08    272
10  Janice Joplette    Snooker Table    08    154
11   Gerald Butters   Massage Room 1    08    153
12       Nancy Dare     Table Tennis    08    143
13     Darren Smith  Badminton Court    08    132
14   Gerald Butters   Tennis Court 1    08    111
15       Tim Boothe   Tennis Court 2    08    109
16     Burton Tracy     Squash Court    08     85
17       Nancy Dare   Massage Room 2    08      9
18     David Pinker       Pool Table    09    408
